# Title: PyTorch_AMP_Training_Loop_with_EarlyStopping
## Description: PyTorch 모델 학습 시 FP16(Mixed Precision)을 사용하여 메모리와 속도를 최적화하고, Early Stopping을 통해 과적합을 방지하는 표준 학습 파이프라인.
## Input: 
 - model (nn.Module): 학습할 PyTorch 모델
 - optimizer (torch.optim): 옵티마이저
 - train_loader (DataLoader): 학습 데이터 로더
 - val_loader (DataLoader): 검증 데이터 로더
 - scheduler (lr_scheduler): 학습률 스케줄러 (Optional)
 - device (torch.device): 학습 장치 (CPU/GPU)
 - epochs (int): 최대 에폭 수
## Output: 
 - best_model (nn.Module): 검증 점수가 가장 높은 모델의 state_dict가 로드된 객체
## Check Point: 
 - `torch.cuda.amp`를 사용하므로 GPU 환경 권장.
 - 평가 지표로 F1 Score(Macro)를 사용하므로 `sklearn` 라이브러리 필요.

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from copy import deepcopy
from tqdm.auto import tqdm
from torch.amp import GradScaler, autocast
from sklearn.metrics import f1_score

# [Block 1] Early Stopping Class
class EarlyStopping:
    """
    검증 손실이 개선되지 않을 때 학습을 조기 종료하고 최적의 모델을 저장하는 유틸리티
    """
    def __init__(self, patience=5, verbose=True, path="best_model_earlystop.pth"):
        self.patience = patience
        self.verbose = verbose
        self.path = path
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.best_model_wts = None

    def __call__(self, score, model):
        if self.best_score is None:
            self.save_checkpoint(score, model)
            self.best_score = score
        elif score <= self.best_score:
            self.counter += 1
            if self.verbose:
                print(f"🔁 EarlyStopping counter: {self.counter} / {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.save_checkpoint(score, model)
            self.best_score = score
            self.counter = 0

    def save_checkpoint(self, score, model):
        if self.verbose:
            if self.best_score is not None and self.best_score != score:
                print(f"Validation score improved ({self.best_score:.4f} --> {score:.4f}). Saving model...")
            else:
                print(f"Validation score set to {score:.4f}. Saving model...")
        torch.save(model.state_dict(), self.path)

# [Block 2] Validation Logic
def validation(model, criterion, val_loader, device):
    """
    검증 데이터셋에 대한 평가를 수행 (AMP 미적용, No Grad)
    """
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(val_loader, desc="Validating", leave=False):
            imgs = imgs.to(device)
            labels = labels.to(device)

            # Inference 시에는 autocast만 적용 (Scaler 불필요)
            with autocast():
                pred = model(imgs)
                loss = criterion(pred, labels)

            # 결과 수집 (GPU -> CPU -> List)
            preds += pred.argmax(1).detach().cpu().numpy().tolist() #각 이미지 확률 분포에서 가장 높은 인덱스 선택 후 list 까지
            true_labels += labels.detach().cpu().numpy().tolist()
            val_loss.append(loss.item())

    _val_loss = np.mean(val_loss)
    _val_score = f1_score(true_labels, preds, average='macro')
    
    return _val_loss, _val_score

# [Block 3] Main Training Loop
def train_pipeline(model, optimizer, train_loader, val_loader, scheduler, device, epochs, early_stopping):
    """
    AMP와 GradScaler를 적용한 메인 학습 루프
    """
    model.to(device)
    
    # Label Smoothing 적용된 Loss 사용 (일반화 성능 향상)
    criterion = nn.CrossEntropyLoss(label_smoothing=0.05).to(device)
    
    # FP16 학습을 위한 Scaler 초기화
    scaler = GradScaler()
    
    for epoch in range(1, epochs + 1):
        model.train()
        train_loss = []
        
        loop = tqdm(train_loader, desc=f"Epoch {epoch}", leave=True)
        for imgs, labels in loop:
            imgs = imgs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad() # [지식] : 파이토치는 기본적으로 gradient를 누적 함.
            # 이를 방지하기 위해 gradient를 초기화

            # Mixed Precision Forward Pass
            with autocast(): # [autocast] : 이 안에서 일어나는 계산은 16비트와 32비트를 섞어서 계산함. --> 메모리 절반으로 줄이기 가능
                outputs = model(imgs)
                loss = criterion(outputs, labels)

            # Scaled Backward Pass ( gradient scaling )
            scaler.scale(loss).backward() # loss 값을 scale 해서 backward 수행 --> gradient vanishing 방지
            scaler.step(optimizer) # 뻥튀기 한 gradient를 다시 원래대로 줄여서 optimizer 업데이트
            scaler.update()

            train_loss.append(loss.item())
            loop.set_postfix(loss=loss.item())

        # Validation Step
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)

        print(f'Epoch [{epoch}], Train Loss: {_train_loss:.5f}, Val Loss: {_val_loss:.5f}, Val F1: {_val_score:.5f}')

        # Scheduler Step
        if scheduler is not None:
            scheduler.step(_val_score)

        # Early Stopping Check
        early_stopping(_val_score, model)
        if early_stopping.early_stop:
            print("Early stopping triggered.")
            break

    # Load Best Model Weights
    model.load_state_dict(torch.load(early_stopping.path))
    return model

## How to Use
1. **데이터 준비**: `CustomDataset`과 `DataLoader`를 사용하여 `train_loader`, `val_loader`를 미리 정의하세요.
2. **모델 및 옵티마이저 정의**: `timm` 등을 사용해 모델을 생성하고 `optimizer`, `scheduler`를 설정하세요.
3. **학습 실행**:
    ```python
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    early_stopping = EarlyStopping(patience=5, path="my_best_model.pth")
    
    trained_model = train_pipeline(
        model=model,
        optimizer=optimizer,
        train_loader=train_loader,
        val_loader=val_loader,
        scheduler=scheduler,
        device=device,
        epochs=100,
        early_stopping=early_stopping
    )
    ```

## Troubleshooting
- **NaN Loss 발생**: `GradScaler` 사용 시 학습률(LR)이 너무 크거나 데이터에 이상치가 있으면 발생할 수 있습니다. `scaler.update()` 전 `torch.nn.utils.clip_grad_norm_`을 추가하거나 LR을 줄이세요.
- **메모리 부족(OOM)**: 배치 사이즈를 줄이거나 `autocast`가 제대로 적용되었는지 확인하세요. `Mixed Precision`은 메모리 사용량을 줄여줍니다.